In [1]:
import json
from web3 import Web3
from bip44 import Wallet
from bip44.utils import get_eth_addr

In [2]:
MNEMONIC = "test test test test test test test test test test test junk"
w = Wallet(MNEMONIC)
sk, pk = w.derive_account("eth", address_index=0)
sender = get_eth_addr(pk)
ZERO_ADDRESS = "0x0000000000000000000000000000000000000000"
ETHER = int(1e18)

# Minimal exec fee that is executed automatically by debridge oracles
DEBRIDGE_EXEC_FEE = int(0.02 * ETHER)

In [3]:
with open('abis/DeBridgeGate.json') as json_file:
    gate_abi = json.load(json_file)

with open('abis/L1Bridge.json') as json_file:
    bridge_abi = json.load(json_file)

In [4]:
kovan_provider = Web3(Web3.HTTPProvider("https://kovan.poa.network"))
#bsc_testnet_provider = Web3(Web3.HTTPProvider("https://data-seed-prebsc-1-s1.binance.org:8545/"))
bsc_testnet_chain_id = 97

In [5]:
gate = kovan_provider.eth.contract(address='0x68D936Cb4723BdD38C488FD50514803f96789d2D', abi=gate_abi['abi'])

In [6]:
global_fixed_nativeFee = gate.functions.globalFixedNativeFee().call()
global_fixed_nativeFee

10000000000000000

In [7]:
bridge = kovan_provider.eth.contract(address='0x279e0cF58d6B3913C917222A61ba9ef5E9fF8057', abi=bridge_abi['abi'])
bridge

In [8]:
bridge.functions.l1Token().call()

'0x55904F2C4B672103a4E04E6BFcb1c5DDC4f41D6F'

In [9]:
# on the destination network (L2Bridge)
l2bridge = "0x463b31B32417f122d25cbAF84Ff263723c11de64"

In [10]:
value = global_fixed_nativeFee + DEBRIDGE_EXEC_FEE
value

30000000000000000

In [11]:
# At this point you should approve token you plan to transfer

In [13]:
tx = bridge.functions.outboundTransfer(sender, 14).buildTransaction({"from": sender, "nonce": kovan_provider.eth.get_transaction_count(sender),
 "gasPrice": 10_000_000_000, "gas": 3_000_000, "value": 50000000000000000 })
tx

{'chainId': 42,
 'from': '0x49518ffe12fCda824FB39CCEa1cceDAf8A724FeB',
 'nonce': 65,
 'gasPrice': 10000000000,
 'gas': 3000000,
 'value': 50000000000000000,
 'to': '0x279e0cF58d6B3913C917222A61ba9ef5E9fF8057',
 'data': '0x8ba9c82800000000000000000000000049518ffe12fcda824fb39ccea1ccedaf8a724feb000000000000000000000000000000000000000000000000000000000000000e'}

In [14]:
raw_tx = kovan_provider.eth.account.sign_transaction(tx, sk).rawTransaction

In [15]:
tx_hash = kovan_provider.eth.send_raw_transaction(raw_tx)
tx_hash

HexBytes('0x268d0ef897868058cd8c7b6547dbe3a5886ec5a37f173f97718e0dbfc1446d49')

In [16]:
print("https://testswap.debridge.finance/transaction?tx=" + tx_hash.hex() + "&chainId=42")

https://testswap.debridge.finance/transaction?tx=0x268d0ef897868058cd8c7b6547dbe3a5886ec5a37f173f97718e0dbfc1446d49&chainId=42


In [17]:
kovan_provider.eth.get_transaction_receipt(tx_hash)

AttributeDict({'blockHash': HexBytes('0x094b050f91f59e7513f4b4fff5bb0a4b3d2cb8af6583613104cb55ae8cfec813'),
 'blockNumber': 29021340,
 'contractAddress': None,
 'cumulativeGasUsed': 196983,
 'effectiveGasPrice': 10000000000,
 'from': '0x49518ffe12fCda824FB39CCEa1cceDAf8A724FeB',
 'gasUsed': 196983,
 'logs': [AttributeDict({'address': '0x55904F2C4B672103a4E04E6BFcb1c5DDC4f41D6F',
   'blockHash': HexBytes('0x094b050f91f59e7513f4b4fff5bb0a4b3d2cb8af6583613104cb55ae8cfec813'),
   'blockNumber': 29021340,
   'data': '0x',
   'logIndex': 0,
   'removed': False,
   'topics': [HexBytes('0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925'),
    HexBytes('0x00000000000000000000000049518ffe12fcda824fb39ccea1ccedaf8a724feb'),
    HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
    HexBytes('0x000000000000000000000000000000000000000000000000000000000000000e')],
   'transactionHash': HexBytes('0x268d0ef897868058cd8c7b6547dbe3a5886ec5a37f173f97718e0dbfc